# EE5295 — One-Click SPICE (RC • VCO • BGR)

Bản này **sanitize** netlist trước khi chạy (loại bỏ dòng `...` gây lỗi), sau đó chạy ngspice ở đúng thư mục để **xuất CSV chắc chắn**.

In [ ]:

import os, shutil, subprocess, pathlib, pandas as pd, matplotlib.pyplot as plt, re

SPICE_DIR = "/huythanh/EE5295_Sim_Scaffold/spice"
RESULTS_DIR = os.path.join(SPICE_DIR, "results")
TMP_DIR = os.path.join(SPICE_DIR, "_tmp_run")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(TMP_DIR, exist_ok=True)

print("SPICE_DIR =", SPICE_DIR)
print("RESULTS_DIR =", RESULTS_DIR)

def which(x):
    from shutil import which as _w
    return _w(x)

assert which("ngspice") is not None, "ngspice chưa có trong PATH"

def sanitize_to_tmp(src_name, tmp_name):
    src = pathlib.Path(SPICE_DIR) / src_name
    tmp = pathlib.Path(TMP_DIR) / tmp_name
    lines = src.read_text().splitlines()
    clean = []
    for ln in lines:
        s = ln.strip()
        if s == "..." or s == "* ..." or s == "* ...":
            continue
        if re.fullmatch(r"[. ]+", s):
            continue
        clean.append(ln)
    tmp.write_text("\n".join(clean) + "\n")
    return tmp

print("✔ Env ready")


## 1) RC Low-Pass — AC sweep & CSV

In [ ]:

from pathlib import Path

rc_tmp = sanitize_to_tmp("example_rc_lp.cir", "example_rc_lp_run.cir")
rc_log = Path(SPICE_DIR) / "rc_ac.log"
rc_csv = Path(RESULTS_DIR) / "rc_bode.csv"

txt = rc_tmp.read_text()
if ".control" not in txt:
    ctrl = """
.control
set wr_singlescale
ac dec 100 10 100Meg
print vdb(out) vp(out)
wrdata results/rc_bode.csv freq vdb(out) vp(out)
meas ac fc WHEN vdb(out)=-3
quit
.endc
"""
    txt = txt.replace(".end", ctrl + "\n.end")
rc_tmp.write_text(txt)

cmd = ["ngspice","-b","-o", str(rc_log.name), str(rc_tmp.name)]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True, cwd=SPICE_DIR)

if rc_csv.exists():
    import pandas as pd, matplotlib.pyplot as plt
    df = pd.read_csv(rc_csv, sep=r"\s+", engine="python")
    print(df.head())
    plt.figure(); plt.semilogx(df["freq"], df["vdb(out)"]); plt.title("RC – Magnitude (dB)"); plt.xlabel("Hz"); plt.ylabel("dB"); plt.show()
    plt.figure(); plt.semilogx(df["freq"], df["vp(out)"]);  plt.title("RC – Phase (deg)");   plt.xlabel("Hz"); plt.ylabel("deg"); plt.show()
else:
    print("⚠️ Không thấy rc_bode.csv — kiểm tra node 'out' trong netlist.")


## 2) Ring VCO — Transient & f–VCTRL

In [ ]:

from pathlib import Path

vco_tmp = sanitize_to_tmp("example_vco_ring.cir", "example_vco_ring_run.cir")
vco_log = Path(SPICE_DIR) / "vco_tran.log"
vco_wave = Path(RESULTS_DIR) / "vco_wave.csv"
vco_fv = Path(RESULTS_DIR) / "vco_fv.csv"

txt = vco_tmp.read_text()
if ".control" not in txt:
    ctrl = """
.control
set wr_singlescale
tran 0.1n 20u 5u uic
meas tran t1 when v(n1) cross=0.6 rise=10
meas tran t2 when v(n1) cross=0.6 rise=11
meas tran per  param = 't2-t1'
meas tran freq param = '1/per'
wrdata results/vco_wave.csv time v(n1)

reset
step param VCTRL list 0.6 0.8 1.0 1.2 1.4
tran 0.1n 20u 5u uic
meas tran t1 when v(n1) cross=0.6 rise=10
meas tran t2 when v(n1) cross=0.6 rise=11
meas tran per  param = 't2-t1'
meas tran freq param = '1/per'
wrdata results/vco_fv.csv freq
quit
.endc
"""
    txt = txt.replace(".end", ctrl + "\n.end")
vco_tmp.write_text(txt)

cmd = ["ngspice","-b","-o", str(vco_log.name), str(vco_tmp.name)]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True, cwd=SPICE_DIR)

if vco_wave.exists():
    import pandas as pd, matplotlib.pyplot as plt
    dfw = pd.read_csv(vco_wave, sep=r"\s+", engine="python")
    print(dfw.head())
    plt.figure(); plt.plot(dfw["time"], dfw["v(n1)"]); plt.title("VCO – Waveform @ n1"); plt.xlabel("Time (s)"); plt.ylabel("V(n1) (V)"); plt.show()
else:
    print("⚠️ Không thấy vco_wave.csv — kiểm tra node 'n1' và thời gian mô phỏng.")

if vco_fv.exists():
    import pandas as pd, matplotlib.pyplot as plt
    dfv = pd.read_csv(vco_fv, sep=r"\s+", engine="python")
    print(dfv.head())
    if "freq" in dfv.columns:
        plt.figure(); plt.plot(dfv.index, dfv["freq"], marker="o"); plt.title("VCO – f vs VCTRL (index=step)"); plt.xlabel("Step"); plt.ylabel("Hz"); plt.show()
else:
    print("⚠️ Không thấy vco_fv.csv — kiểm tra param VCTRL trong netlist.")


## 3) BGR — Vref vs VDD (Line Regulation) + Temp logs

In [ ]:

from pathlib import Path

bgr_tmp = sanitize_to_tmp("example_bgr_skeleton.cir", "example_bgr_skeleton_run.cir")
bgr_log = Path(SPICE_DIR) / "bgr_dc.log"
bgr_line = Path(RESULTS_DIR) / "bgr_line.csv"

txt = bgr_tmp.read_text()
if ".control" not in txt:
    ctrl = """
.control
set wr_singlescale
op
print v(vref)
reset
dc vdd 1.2 2.0 0.05
wrdata results/bgr_line.csv vdd v(vref)

reset
set temp=-40
op
print "TEMP=-40C Vref=" v(vref)
set temp=27
op
print "TEMP=27C Vref=" v(vref)
set temp=85
op
print "TEMP=85C Vref=" v(vref)
quit
.endc
"""
    txt = txt.replace(".end", ctrl + "\n.end")
bgr_tmp.write_text(txt)

cmd = ["ngspice","-b","-o", str(bgr_log.name), str(bgr_tmp.name)]
print("Running:", " ".join(cmd))
subprocess.run(cmd, check=True, cwd=SPICE_DIR)

if bgr_line.exists():
    import pandas as pd, matplotlib.pyplot as plt
    dfl = pd.read_csv(bgr_line, sep=r"\s+", engine="python")
    print(dfl.head())
    plt.figure(); plt.plot(dfl["vdd"], dfl["v(vref)"], marker="o"); plt.title("BGR – Vref vs VDD"); plt.xlabel("VDD (V)"); plt.ylabel("Vref (V)"); plt.show()
else:
    print("⚠️ Không thấy bgr_line.csv — kiểm tra tên nguồn 'vdd' và node 'vref'.")


## 4) ZIP kết quả

In [ ]:

import os, zipfile
zip_path = os.path.join(SPICE_DIR, "results_spice.zip")
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for name in os.listdir(RESULTS_DIR):
        z.write(os.path.join(RESULTS_DIR, name), arcname="results/"+name)
    for log in ["rc_ac.log","vco_tran.log","bgr_dc.log"]:
        p = os.path.join(SPICE_DIR, log)
        if os.path.isfile(p):
            z.write(p, arcname="logs/"+os.path.basename(p))
print("ZIP created:", zip_path)
